In [ ]:
import specpy

from pipeline2.callback_buildingblocks.coordinate_value_wrappers import StageOffsetsSettingsGenerator
from pipeline2.callback_buildingblocks.regular_position_generators import PositionListOffsetGenerator
from pipeline2.callback_buildingblocks.parameter_filtering import DefaultLocationRemover, DefaultLocationKeeper
from pipeline2.callback_buildingblocks.static_settings import FOVSettingsGenerator, ScanModeSettingsGenerator
from pipeline2.callback_buildingblocks.data_selection import NewestDataSelector, NewestSettingsSelector
from pipeline2.callback_buildingblocks.static_settings import JSONSettingsLoader
from pipeline2.detection.clasification_check import AcceptanceCheck
from pipeline2.utils.tiling import centered_tiles
from pipeline2.taskgeneration import AcquisitionTaskGenerator
from pipeline2.pipeline import AcquisitionPipeline
from pipeline2.imspector import get_current_stage_coords

In [ ]:
def area_above_threshold_check(img, threshold=10, min_area=25):
    """simple check if enough pixels above background are in an image"""
    return (img >= threshold).sum() >= min_area

In [ ]:
# get current image from Imspector to test
im = specpy.get_application()
ms = im.active_measurement()
img = ms.stack(0).data().squeeze()

area_above_threshold_check(img, threshold=100)

In [ ]:
# where to save & whether to save combined HDF5 file
save_folder = 'D:/AUTOMATION/TEST/20241010_selective_overviews'
save_hdf5 = False

# path of measurement parameters (dumped to JSON file)
measurement_parameters = 'test_configs/20241010_overview_3d_640.json'

# yx FOV size
fov_size = [50e-6, 50e-6]

# (larger) pixel sizes for pre_scan
pre_scan_pixelsizes = [0.25e-6, 0.5e-6, 0.5e-6]

# scan mode of pre-scan
# 'xy' to only do one plane
pre_scan_mode = 'xy'

# parameter values for area_above_threshold_check
acceptance_parameters = {'threshold':50, 'min_area':50}

# yx number of tiles
n_tiles = [5, 5]

# how much the tiles should overlap (0-1)
overlap_fraction = 0

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

# generate regular grid around current stage position
coordinate_list = centered_tiles(start_coords, length=[0]+fov_size, n_tiles=[1]+n_tiles, overlap=overlap_fraction)

# show for verification
coordinate_list

In [ ]:
# build pipeline object (2 levels: 'pre_scan', 'field')
pipeline = AcquisitionPipeline('multipoint-acquisition', save_folder, ['pre_scan', 'field'], save_combined_hdf5=save_hdf5)

# field will re-use the index of the associated pre-scan
pipeline._levels_reusing_parent_index = ['field']
# pre-scan index will not show up in filename (only for actual pre-scan images)
pipeline._masked_levels_in_filename = ['pre_scan']

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('pre_scan',
    DefaultLocationRemover(JSONSettingsLoader(measurement_parameters, as_measurements=False)),
    ScanModeSettingsGenerator(pre_scan_mode),
    FOVSettingsGenerator(pixel_sizes=pre_scan_pixelsizes),
    StageOffsetsSettingsGenerator(PositionListOffsetGenerator(coordinate_list))
)

# callback to create a measurement with same location, but larger pixel size / 3D
# if something is detected in pre-scan via AcceptanceCheck
accept_position_callback = AcquisitionTaskGenerator('field',
    DefaultLocationRemover(JSONSettingsLoader(measurement_parameters, as_measurements=False)),
    DefaultLocationKeeper(NewestSettingsSelector(pipeline, 'pre_scan')),
    AcceptanceCheck(NewestDataSelector(pipeline, 'pre_scan'), area_above_threshold_check, check_function_kwargs=acceptance_parameters)
)

# attach callback so that after each position, pre_scan the next one will be enqueued
pipeline.add_callback(next_position_generator, 'pre_scan')

# attach callback so that after each pre-scan, a more detailled stack will be images, if accepted
pipeline.add_callback(accept_position_callback, 'pre_scan')

# start with initial task from callback
pipeline.run(next_position_generator)